In [149]:
from DatabaseConnections import *
import params
import pandas as pd
import cPickle as cpickle
import itertools
import math
import os
import csv
from operator import itemgetter
from IPython.display import clear_output
# clear_output()

In [20]:
db = PostgresDb(params.DB_HOST, params.DB_NAME) 

User name (RIS_TRAINING):shostetter
Password (RIS_TRAINING):········


### Import Roadbed Pointer List into database

In [152]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        read_data = f.read()
    return read_data

def split_to_rows(raw_data):
    d = raw_data.split('\n')
    output = []
    for row in d:
        output.append(row.split(','))
    return output

def split_to_columns(data_rows):
    data = []
    # ['RPL_ID', 'SegmentIDG', 'SegmentIDR', 'RPC', 'NCI',
    #         'NodeLevelF', 'NodeLevelT', 'R_FrNd', 'G_FrNd', 'R_ToNd', 'G_ToNd']
    idn = 0
    for row in data_rows:
        if row != ['']:
            # print row
            idn += 1
            g_seg = row[0][:7]
            g_seg_typ = row[0][7]
            r_seg = row[0][8:15]
            rpc = row[0][16]
            nci = row[0][18]
            fnode_level_code = row[0][22]
            tnode_level_code = row[0][26]
            f_node_rb_seg = row[0][28:35]
            f_node_g_seg = row[0][36:43]
            t_node_rb_seg = row[0][44:51]
            t_node_g_seg = row[0][52:59]
            data.append([idn, g_seg, r_seg, rpc, nci, fnode_level_code, tnode_level_code,
                         f_node_rb_seg, f_node_g_seg, t_node_rb_seg, t_node_g_seg])
    return data

def add_to_db(db, rpl, data):
    # make sure table exists and is clean
    db.query("""DROP TABLE if exists {0};
                    CREATE TABLE {0}
                    (
                      rpl_id bigint, segmentidg bigint, segmentidr bigint,
                      rpc character varying(1), nci character varying(1),
                      nodelevelf character varying(1), nodelevelt character varying(1),
                      r_frnd bigint, g_frnd bigint, r_tond bigint, g_tond bigint
                    );
                """.format(rpl))
    for row in data:
        if row[0] != 'RPL_ID':
            add_row(db, row)


def add_row(db, row):
    if row[0]:
        db.query("""INSERT INTO tbl_rpl (rpl_id, segmentidg, segmentidr, rpc, nci,nodelevelf,
                        nodelevelt, r_frnd, g_frnd, r_tond, g_tond)
                        VALUES (%s);
                    """ % str(row)[1:-1])
        print 'Added %s' % row[0]
        clear_output()

In [153]:
def add_rpl(db):
    r_data = read_file(os.path.join(params.FOLDER, params.RPL_TXT))
    dta = split_to_rows(r_data)
    split_data = split_to_columns(dta)
    add_to_db(db, params.RPL, split_data)

add_rpl(db)
query_to_table(db, "select * from {} limit 10".format(params.RPL))

,rpl_id,segmentidg,segmentidr,rpc,nci,nodelevelf,nodelevelt,r_frnd,g_frnd,r_tond,g_tond
0,1,603,152742,R,B,,,87191,436,87224,427
1,2,603,143668,L,B,,,94522,436,94515,427
2,3,611,152783,R,B,,,86306,446,87191,436
3,4,611,143667,L,B,,,94529,446,94522,436
4,5,668,152704,R,B,,,87106,492,87107,494
5,6,668,152780,L,B,,,87138,492,88593,494
6,7,670,152859,R,B,,,87107,494,88567,497
7,8,670,150838,L,B,,,88593,494,496,497
8,9,674,150841,R,B,,,88567,497,88590,498
9,10,674,144716,L,B,,,496,497,86315,498


In [150]:
@timeDec
def save_dictionaries(io='OUT'):
    """write out the data in dictionaries, may only be useful for testing..."""
    if io == 'OUT':
        cpickle.dump(params.nodeStreetNames, open(os.path.join(params.FOLDER, "nodeStreetNames.p"), "wb"))
        cpickle.dump(params.nodeIsIntersection, open(os.path.join(params.FOLDER, "nodeIsIntersection.p"), "wb"))
        cpickle.dump(params.nodeNextSteps, open(os.path.join(params.FOLDER, "nodeNextSteps.p"), "wb"))
        cpickle.dump(params.segmentBlocks, open(os.path.join(params.FOLDER, "segmentBlocks.p"), "wb"))
        cpickle.dump(params.node_master, open(os.path.join(params.FOLDER, "nodeMaster.p"), "wb"))
        cpickle.dump(params.clusterIntersections, open(os.path.join(params.FOLDER, "clusterIntersections.p"), "wb"))
        cpickle.dump(params.streetSet, open(os.path.join(params.FOLDER, "streetSet.p"), "wb"))
        cpickle.dump(params.mft1Dict, open(os.path.join(params.FOLDER, "mft1Dict.p"), "wb"))
        cpickle.dump(params.altGraph, open(os.path.join(params.FOLDER, "altGraph.p"), "wb"))
        cpickle.dump(params.mfts, open(os.path.join(params.FOLDER, "mfts.p"), "wb"))
        cpickle.dump(params.coordFromMaster, open(os.path.join(params.FOLDER, "coordFromMaster.p"), "wb"))
    else:
        d1 = cpickle.load(open(os.path.join(params.FOLDER, "nodeStreetNames.p"), "rb"))
        d2 = cpickle.load(open(os.path.join(params.FOLDER, "nodeIsIntersection.p"), "rb"))
        d3 = cpickle.load(open(os.path.join(params.FOLDER, "nodeNextSteps.p"), "rb"))
        d4 = cpickle.load(open(os.path.join(params.FOLDER, "segmentBlocks.p"), "rb"))
        d5 = cpickle.load(open(os.path.join(params.FOLDER, "nodeMaster.p"), "rb"))
        d6 = cpickle.load(open(os.path.join(params.FOLDER, "clusterIntersections.p"), "rb"))
        d7 = cpickle.load(open(os.path.join(params.FOLDER, "streetSet.p"), "rb"))
        d8 = cpickle.load(open(os.path.join(params.FOLDER, "mft1Dict.p"), "rb"))
        d9 = cpickle.load(open(os.path.join(params.FOLDER, "altGraph.p"), "rb"))
        d10 = cpickle.load(open(os.path.join(params.FOLDER, "mfts.p"), "rb"))
        d11 = cpickle.load(open(os.path.join(params.FOLDER, "coordFromMaster.p"), "rb"))
        return d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11

In [4]:
def reset(db, version):
    tables = ['node_stnameft', 'lion', 'node', 'c_master_node',
              'c_master_segs', 'C_Lion_%s' % version, 'C_Lion_Nodes_%s' % version, 'm']
    for t in tables:
        print 'Droping %s\n' % t
        db.query("DROP TABLE if exists %s" % t)
    return raw_input('Re-imported lion and nodes? <enter>\n').upper()

In [5]:
# clean out old versions of lion ?

reset(db, params.VERSION)


Droping node_stnameft

Update sucessfull
Droping lion

Update sucessfull
Droping node

Update sucessfull
Droping c_master_node

Update sucessfull
Droping c_master_segs

Update sucessfull
Droping C_Lion_16d

Update sucessfull
Droping C_Lion_Nodes_16d

Update sucessfull
Droping m

Update sucessfull
Re-imported lion and nodes? <enter>



''

# Preprocess 
## Set up data schema

In [8]:
@timeDec
def pre_process(db, lion, node, version, highways=False):
    print 'Adding version number (%s)...\n' % version
    db.query("alter table %s add version varchar(50)" % lion)
    db.query("alter table %s add version varchar(50)" % node)
    db.query("update %s set version  = '%s'" % (lion, version))
    db.query("update %s set version  = '%s'" % (node, version))
    
    print 'Adding master ID fields to tables...\n'
    db.query("alter table %s add column mft int, add column masteridfrom int, add column masteridto int" % lion)
    db.query("alter table %s add column masterid int" % node)
    db.query("alter table %s add column is_int int" % node)
    
    print 'Creating node street name table (highways = %s)...\n' % str(highways)
    if highways:
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master from (
                        select nodeidfrom as node, street
                        from {0}
                            where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and
                            (nonped = 'D' or nonped is null)
                            and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and (nonped = 'D'
                        or nonped is null)
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidto, street

                        union
                        /* ---------- ADD BACK IN PELHAM PKWY MAIN LINE ---------- */
                        select nodeidfrom as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and
                        (nonped = 'V' and street = 'PELHAM PARKWAY')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street
                        /* ---------- ADD BACK IN PELHAM PKWY MAIN LINE ---------- */
                        union
                        /* ---------- ADD BACK IN FORDHAM RD MAIN LINE ---------- */
                        select nodeidfrom as node, street
                        from {0}
                        where nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'
                        group by nodeidfrom, street
                        /* ---------- ADD BACK IN FORDHAM RD MAIN LINE ---------- */
                        ) ft
                    group by node, street)
                """.format(lion, 'PED OVPS'))
    else:
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master
                    from (
                        select nodeidfrom as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidto, street
                        ) ft
                    group by node, street)
                """.format(lion, 'PED OVPS'))

        
# Run preprocess
pre_process(db, params.LION, params.NODE, params.VERSION, params.HIGHWAYS)
# Check resutls


Adding version number (16d)...

Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Adding master ID fields to tables...

Update sucessfull
Update sucessfull
Update sucessfull
Creating node street name table (highways = True)...

Update sucessfull
'pre_process' 59.67 sec


In [9]:
query_to_table(db, "select * from node_stnameFT limit 10")

,node,street,master
0,13,SHORE ROAD,0
1,14,PERTH AMBOY PLACE,0
2,14,WARDS POINT AVENUE,0
3,16,WARDS POINT AVENUE,0
4,17,SATTERLEE STREET,0
5,18,SATTERLEE STREET,0
6,18,SHORE ROAD,0
7,19,PERTH AMBOY PLACE,0
8,19,SATTERLEE STREET,0
9,21,AMBOY ROAD,0


# Part 2
### Build data dictionaries

In [10]:
def de_suffix(name):  # todo fix e as part of east example
    """cleans up street names for less percise matches"""
    d = {' WEST': ' DIR', ' EAST': ' DIR',
         ' SOUTH': ' DIR', ' NORTH': ' DIR',
         ' EXIT': ' DIR', ' ENTRANCE': ' DIR',
         ' APPROACH': ' DIR', ' NORTHBOUND': ' DIR',
         ' SOUTHBOUND': ' DIR', ' EASTBOUND': ' DIR',
         ' WESTBOUND': ' DIR'}  # new needs to be tested!!!!
    for i in d:
        if name not in ('WEST STREET', 'SOUTH STREET', 'NORTH STREET', 'EAST STREET',
                        'WEST AVENUE', 'SOUTH AVENUE', 'NORTH AVENUE', 'EAST AVENUE',
                        'WEST BOULEVARD', 'SOUTH BOULEVARD', 'NORTH BOULEVARD', 'EAST BOULEVARD',
                        'WEST LOOP', 'SOUTH LOOP', 'NORTH LOOP', 'EAST LOOP',
                        'WEST DRIVE', 'SOUTH DRIVE', 'NORTH DRIVE', 'EAST DRIVE',
                        'WEST ROAD', 'SOUTH ROAD', 'NORTH ROAD', 'EAST ROAD',
                        'JUNIPER BOULEVARD NORTH', 'JUNIPER BOULEVARD SOUTH'
                        'PROSPECT PARK WEST', 'AVENUE N', 'AVENUE S', 'AVENUE E',
                        'AVENUE W',):
            name = name.replace(i, d[i])
    return name 
    
@timeDec
def node_names(db, nodeStreetNames):
    data = db.query("select * from node_stnameFT")
    for row in data[0]:
        node, street = row[0], de_suffix(row[1])
        if node not in nodeStreetNames.keys():
            nodeStreetNames[node] = set([street])
        else:
            nodeStreetNames[node].add(street)
    return nodeStreetNames

params.nodeStreetNames = node_names(db, params.nodeStreetNames)

'node_names' 100.80 sec


In [11]:
@timeDec
def get_double_blocks(db, lion, node_table, highways=False):
    # get problem nodes where the blocks involved have more than 1 street name
    if highways:
        h = '''and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                or (nonped = 'V' and rb_layer in ('B', 'G')
                and street = 'EAST FORDHAM ROAD'))'''  # ADD BACK IN PELHAM PKWY
    else:
        h = ''
    db.query("""drop table if exists doubles; 
                create table doubles as (
                    select nf::int, nt::int, count(street)
                    from (
                        select  nodeidfrom as nf, nodeidto as nt, street
                        from {0}
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') {1}
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by  nodeidfrom, nodeidto, street
                    union
                        select  nodeidto as nf, nodeidfrom as nt, street
                        from {0}
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') {1}
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by  nodeidfrom, nodeidto, street
                    ) as t
                    group by nf, nt
                    having count(street) > 1)""".format(lion, h, 'PED OVPS'))
    
    # then those nodes need to have > 2 streets to be an intersection. 
    db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt from (
                            select nodeid, is_int, street
                            from (
                                    select nodeid, is_int, street
                                    from {0} 
                                    join {1} on st_dwithin({0}.geom, {1}.geom,1)
                                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') {2}
                                    and street != 'UNNAMED STREET' and street not like '%{3}%'
                                    group by nodeid, is_int, street
                            ) as base, 
                    doubles
                    where nodeid = nf or nodeid = nt
                            group by nodeid, is_int, street
                    ) as grp group by nodeid, is_int
                            having count(*) < 3
                );""".format(node_table, lion, h, 'PED OVPS'))
    # update revised nodes
    db.query("""update %s as n
                    set is_int = Null
                    from temp_int
                    where n.nodeid =temp_int.nodeid
                    and n.is_int = -1
                    ;""" % node_table)
    db.query("drop table temp_int; drop table doubles;")

@timeDec
def intersection_binary(db, node_table, lion, nodeIsIntersection, highways=False):
    """Updates the is_int field in db"""
    if highways:
        db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt
                    from (
                        select nodeid, is_int, street
                        from {0} 
                        join {1} on st_dwithin({0}.geom, {1}.geom,1)
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') and 
                        (
                            (
                                nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY')
                                or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD')
                            )
                        )--for highway exclution
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by nodeid, is_int, street
                    ) as base
                    group by nodeid, is_int
                    having count(*) >1
                );""".format(node_table, lion, 'PED OVPS'))
    else:
        db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt
                    from (
                        select nodeid, is_int, street
                        from {0} 
                        join {1} on st_dwithin({0}.geom, {1}.geom,1)
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by nodeid, is_int, street
                    ) as base
                    group by nodeid, is_int
                    having count(*) >1
                );""".format(node_table, lion, 'PED OVPS'))

    db.query("update {0} set is_int = -1 from temp_int where {0}.nodeid=temp_int.nodeid;".format(node_table))
    db.query("drop table temp_int;")


    # fix locations with double segments...
    get_double_blocks(db, lion, node_table, highways)  # 149.80 sec

    print 'DB updated'
    # get data from db into local dictionary
    data = db.query("select nodeid, is_int from %s " % node_table)  # where is_int =-1
    for i in data[0]:
        if i[1]:
            nodeIsIntersection[int(i[0])] = True
        else:
            nodeIsIntersection[int(i[0])] = False
    del data
    print 'Dictionary updated'
    return nodeIsIntersection

params.nodeIsIntersection = intersection_binary(db, params.NODE, params.LION, params.nodeIsIntersection, params.HIGHWAYS)

Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
'get_double_blocks' 22.97 sec
DB updated
Dictionary updated
'intersection_binary' 61.05 sec


In [12]:
def add_to_graph(street, node1, node2, nodeNextSteps):
    """checks if segment is in the graph adds it if not"""
    if node1 not in nodeNextSteps.keys():
        nodeNextSteps[node1] = {street: set([node2])}
    else:
        if street not in nodeNextSteps[node1].keys():
            nodeNextSteps[node1][street] = set([node2])
        else:
            nodeNextSteps[node1][street].add(node2)
    return nodeNextSteps


@timeDec
def graph(db, lion, nodeNextSteps, highways=False):
    """Graph the street network"""
    if highways:
        segs = db.query("""select street, segmentid, nodeidfrom, nodeidto 
                        from {0} 
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') and
                        ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                        or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                    """.format(lion, 'PED OVPS'))
    else:
        segs = db.query("""select street, segmentid, nodeidfrom, nodeidto 
                        from {0} 
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                    """.format(lion, 'PED OVPS'))

    print 'Done with DB'
    for row in segs[0]:
        street, segmentid, nodeidfrom, nodeidto = row[0], row[1], int(row[2]), int(row[3])
        nodeNextSteps = add_to_graph(street, nodeidfrom, nodeidto, nodeNextSteps)
        nodeNextSteps = add_to_graph(street, nodeidto, nodeidfrom, nodeNextSteps)
    del segs
    return nodeNextSteps


params.nodeNextSteps = graph(db, params.LION, params.nodeNextSteps, params.HIGHWAYS)  # 179.22 sec

Done with DB
'graph' 198.38 sec


### Break point - save progress

In [151]:
save_dictionaries('OUT')  # 19.72 sec
# params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 sec

'save_dictionaries' 6.06 sec


# Part 3
### Build simple blocks

In [4]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 1.95 sec


In [5]:
# simple graph search for grouping blocks

def go_to_end(street, done, todo, nodeIsIntersection=params.nodeIsIntersection, nodeNextSteps=params.nodeNextSteps):
    # anything left in the queue?
    if len(todo) == 0:
        # you're done exit
        return done
    else:
        # get new starting point
        start = todo.pop()
        # add it to the visited nodes
        done.append(start)
        # make sure it is not an ending point
        if not nodeIsIntersection[start]:
            # see where we can go from here
            for i in nodeNextSteps[start][street]:
                # make sure we haven't done it yet
                if i not in done:
                    # add it to the queue
                    todo.add(i)
        return go_to_end(street, done, todo)
    
@timeDec
def search(nodeStreetNames, streetSet, nodeIsIntersection, nodeNextSteps):
    """gets intersection to intersection segments or collection of segments"""
    for startNode in nodeStreetNames.keys():
        if nodeIsIntersection[startNode]:
            for street in nodeNextSteps[startNode].keys():
                queue = list(nodeNextSteps[startNode][street])
                while len(queue) > 0:
                    to_node = queue.pop()
                    streetSet.append(go_to_end(street, [startNode], set([to_node])))
    return nodeStreetNames, streetSet

params.nodeStreetNames, params.streetSet = search(params.nodeStreetNames, params.streetSet, params.nodeIsIntersection, params.nodeNextSteps)

print '\nnodeStreetNames(dict) sample: {{{0} : {1}}}'.format(params.nodeStreetNames.keys()[50], 
                                                             params.nodeStreetNames[params.nodeStreetNames.keys()[50]])

'search' 0.59 sec

nodeStreetNames(dict) sample: {52 : set(['CRAIG AVENUE', 'HYLAN BOULEVARD'])}


In [6]:
def write_csv(out_file, data_to_write, header=None):
    row_cnt = 0
    with open(out_file, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        if header:
            writer.writerow(header)
        for row in data_to_write:
            writer.writerow(row)  # this writes the rows to the csv file row needs to be a list
            row_cnt += 1
    return str(row_cnt) + " rows were written to " + str(out_file)

In [7]:
@timeDec
def mft_round1(db, lion, altGraph, streetSet, mft1Dict, folder, highways=False):
    """gets simplest master segments based on from to nodes, intersections only"""
    if highways:
        sql = """select segmentid, nodeidfrom, nodeidto
                from {0} 
                where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                and street != 'UNNAMED STREET' and street not like '%{1}%'
                group by segmentid, nodeidfrom, nodeidto""".format(lion, 'PED OVPS')
    else:
        sql = """select segmentid, nodeidfrom, nodeidto
                from {0}  
                where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                and street != 'UNNAMED STREET' and street not like '%{1}%'
                group by segmentid, nodeidfrom, nodeidto""".format(lion, 'PED OVPS')
    data = db.query(sql)
    # build local dict of seg node ids from-to to build mft
    for row in data[0]:
        # build up simple graph
        # node : [[other node , segid], [other node , segid]]
        segment, fromid, toid = row
        if int(fromid) not in altGraph.keys():
            altGraph[int(fromid)] = [[int(toid), segment]]
        else:
            altGraph[int(fromid)].append([int(toid), segment])

        if int(toid) not in altGraph.keys():
            altGraph[int(toid)] = [[int(fromid), segment]]
        else:
            altGraph[int(toid)].append([int(fromid), segment])
    print 'done graphing\n'
    mft = 0
    to_write_out = []
    for block in streetSet:  # streetSet is a set of nodes that together make up a block
        if block:
            print mft
            clear_output()
            mft += 1
            for node in block:
                for pair in altGraph[node]:
                    if pair[0] in block:  # both ends of the street are in the block
                        if mft not in mft1Dict.keys():
                            mft1Dict[mft] = []
                        mft1Dict[mft].append(pair[1])
                        to_write_out.append([mft, pair[1]])
    print 'Writing out csv\n'
    write_csv(os.path.join(folder, "mft.csv"), to_write_out, ['mft', 'segment'])  
    db.query('drop table if exists tempMaster; CREATE TABLE tempMaster (mft varchar(10), seg varchar(10))')
    return altGraph

params.altGraph = mft_round1(db,  params.LION, params.altGraph,  params.streetSet, params.mft1Dict, 
                             params.FOLDER, params.HIGHWAYS)  # 1367.67 sec

print '\naltGraph(dict) sample: {{{0} : {1}}}'.format(params.altGraph.keys()[50], 
                                                             params.nodeStreetNames[params.altGraph.keys()[50]])

Writing out csv

Update sucessfull
'mft_round1' 1328.75 sec

altGraph(dict) sample: {51 : set(['HYLAN BOULEVARD', 'SATTERLEE STREET'])}


In [8]:
raw_input("""PG Console \n\n \copy tempMaster FROM '%s\mft.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % params.FOLDER)
clear_output()

In [10]:
@timeDec
def add_temp_table(db, lion):  # Assumes that tempMaster is imported
    db.query("""update %s as l
                    set mft = t.mft::int
                    from tempmaster as t
                    where l.segmentid =t.seg""" % lion)
    db.query("drop table tempmaster")

add_temp_table(db, params.LION)  # 35.44 sec

Update sucessfull
Update sucessfull
'add_temp_table' 135.45 sec


In [11]:
save_dictionaries('OUT')  # 10.07 sec

'save_dictionaries' 4.44 sec


# Part 4
### Build simple intersections

In [12]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 4.78 sec


In [14]:
@timeDec
def add_master_place_to_street_names(nodeStreetNames, nodeIsIntersection):
    to_remove = []
    for n in nodeStreetNames:
        if nodeIsIntersection[n]:  # only want intersection nodes
            # reformat the dict to have a spot for master id
            nodeStreetNames[n] = [nodeStreetNames[n], 0]
        else:
            # store non-intersections to remove latter
            to_remove.append(n)
    for i in to_remove:
        # not intersection remove it from dictionary
        del nodeStreetNames[i]
    return nodeStreetNames

params.nodeStreetNames = add_master_place_to_street_names(params.nodeStreetNames, params.nodeIsIntersection)

'add_master_place_to_street_names' 15.44 sec


In [16]:
def street_name_key(street_set):
    street_set = list(street_set)
    street_set.sort()
    street_set = str(street_set)
    return street_set[1:-1]

@timeDec
def intersection_cluster_dict(nodeStreetNames, clusterIntersections):
    # get street names by intersetion
    for node in nodeStreetNames.keys():
        # order the street names
        street_key = street_name_key(nodeStreetNames[node][0])
        # if not in dictionary, add street name set
        if street_key not in clusterIntersections.keys():
            clusterIntersections[street_key] = [set(), 0]
        # add node to the street name set dict
        clusterIntersections[street_key][0].add(node)
    return nodeStreetNames

params.nodeStreetNames = intersection_cluster_dict(params.nodeStreetNames, params.clusterIntersections)

'intersection_cluster_dict' 30.17 sec


In [18]:
@timeDec
def master_intersection_first_pass(clusterIntersections, node_master):
    master = 0
    for street_set in clusterIntersections.keys():
        master += 1
        # add masterid to each set of street names
        clusterIntersections[street_set][-1] = master
        # update node: master dict
        for node in clusterIntersections[street_set][0]:
            node_master[node] = master
    return clusterIntersections, node_master

params.clusterIntersections, params.node_master = master_intersection_first_pass(params.clusterIntersections, params.node_master)

'master_intersection_first_pass' 0.04 sec


In [27]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [33]:
@timeDec
def check_distance(db, node_table, folder, clusterIntersections):
    tolerance = 1000  # used to be 300.........
    data = db.query("select nodeid, st_x(geom), st_y(geom) from %s where is_int=-1" % node_table)
    loc = {}
    # get geometry
    for row in data[0]:
        node, x, y = row
        loc[node] = [x, y]
    # save the loc table for later
    cpickle.dump(loc, open(os.path.join(folder, "loc.p"), "wb"))

    problems = []
    for master in clusterIntersections.keys():
        # look for sets 2 nodes
        if len(clusterIntersections[master][0]) > 1:
            # get node's geom and check dist
            # n1, n2 = list(clusterIntersections[master][0])[:2]
            for pair in itertools.product(list(clusterIntersections[master][0]), repeat=2):
                n1, n2 = pair
                if distance(loc[n1][0], loc[n1][1], loc[n2][0], loc[n2][1]) > tolerance:
                    problems.append(master)
    return problems

problems = check_distance(db, params.NODE, params.FOLDER, params.clusterIntersections)
# sample
query_to_table(db, "select nodeid, st_x(geom), st_y(geom) from %s where is_int=-1 limit 10" % params.NODE)

'check_distance' 0.67 sec


,nodeid,st_x,st_y
0,45,914808.852808,121744.432234
1,47,914832.253351,121750.466670
2,49,914955.193389,121071.348636
3,51,914209.938678,122562.903986
4,52,914345.791747,122589.380310
5,53,914353.677558,122374.442429
6,54,914580.200717,122647.926778
7,55,914808.015212,122709.386626
8,56,914081.533757,123595.818541
9,57,914321.428280,123638.846669


In [37]:
# @timeDec
def update_problem_groups(problems, neighbors, node_master):
    for problem in problems:
        pcts = problem.keys()
        # simple case
        if len(pcts) == 2:
            if pcts[1] not in neighbors[pcts[0]]:  # pcts are not neighbors so update
                mx = get_max_master(node_master)
                for node in problem[pcts[1]]:
                    node_master[node] = mx + 1
                    # complex case with more than 2 pcts for now assume its a problem and update
                    # come back to this if becomes messy
        for pct in pcts:
            mx = get_max_master(node_master)
            for node in problem[pct]:
                node_master[node] = mx + 1
    return node_master

def get_max_master(node_master):
    # returns the largest master nodeid in the dict
    mx_master = 0
    for n in node_master.keys():
        if node_master[n] > mx_master:
            mx_master = node_master[n]
    return mx_master

@timeDec
def check_problem_pcts(db, problems, node_table, lion_table, precinct_table, clusterIntersections, node_master):
    # make lookup table
    db.query("drop table if exists c_intersection_name")
    # this fails if all the streets' connect to the node are "from" or if all the streets' connect to the node are "to"
    db.query("""create table c_intersection_name as (
                    select nodeid::int as nodeid, precinct, l.street as street1, ll.street as street2
                    from {0} as n
                    join {1} as l on l.nodeidfrom::int=nodeid::int
                    join {1} as ll on ll.nodeidto::int=nodeid::int
                    join {2} as p on st_within(n.geom, p.geom) 
                    where l.street !=ll.street
                    )""".format(node_table, lion_table, precinct_table))
    # store locally
    lookup = {}
    # sample 
    print query_to_table(db, "select nodeid, precinct from c_intersection_name limit 10")
    
    data = db.query("select nodeid, precinct from c_intersection_name")
    for row in data[0]:
        node, pct = row
        lookup[node] = pct
    # check if precincts are neighbors (boundary issues)
    data = db.query("""select p1.precinct, p2.precinct
                    from {0} as p1
                    join {0} as p2
                    on st_intersects(p1.geom, p2.geom)
                    """.format(precinct_table))
    neighbors = {}
    for row in data[0]:
        p1, p2 = row
        if p1 not in neighbors:
            neighbors[p1] = set()
        neighbors[p1].add(p2)

    to_update = []
    for problem in problems:
        rev_lookup = {}
        for node in clusterIntersections[problem][0]:
            if node in lookup.keys():  # known failure for 62493 all streets are "from"
                if lookup[node] not in rev_lookup:
                    rev_lookup[lookup[node]] = set()
                rev_lookup[lookup[node]].add(node)
        if len(rev_lookup) > 1:
            to_update.append(rev_lookup)
    node_master = update_problem_groups(to_update, neighbors, node_master)
    return node_master

params.node_master = check_problem_pcts(db, problems, params.NODE, params.LION, 
                                        params.PRECINCTS, params.clusterIntersections, params.node_master)

Update sucessfull
Update sucessfull
   nodeid precinct
0   13404       62
1   13404       62
2    2131      123
3   14679       76
4   21626       20
5   21626       20
6   29639       75
7   65562       94
8   47748       42
9   47845       42
'check_problem_pcts' 48.84 sec


In [38]:
save_dictionaries('OUT')  # 10.07 sec

'save_dictionaries' 5.90 sec


# Part 5
### Build simple intersections 2 near by nodes

In [39]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 6.39 sec


In [43]:
def near_nodes(db, node_table, search_distance, node):
    # get nodes within search distance of a given node
    nds = db.query("""select nodeid::int
                    from {0} as nn,
                    (
                            select geom 
                            from {0} 
                            where nodeid = {1}
                    ) as base
                    where st_dwithin(nn.geom, base.geom, {2})
                    and nodeid != {1}
                    and is_int = -1
                    """.format(node_table, node, search_distance))
    return [n[0] for n in nds[0]]

def get_best_master(node_master, master1, master2):
    # find the master with more nodes already associated with it
    mm = {master1: 0, master2: 0}
    for node in node_master.keys():
        if node_master[node] == master1:
            mm[master1] += 1
        elif node_master[node] == master2:
            mm[master2] += 1
        else:
            pass
    if mm[master1] > mm[master2]:
        return master1
    else:
        return master2

@timeDec
def near_by_simple(db, node_table, node_master, search_distance):
    # get all nodes within seach distance
    for node in node_master.keys():
        n_nodes = near_nodes(db, node_table, search_distance, node)
        for n in n_nodes:
            # go through those nodes
            new_master = get_best_master(node_master, node_master[n], node_master[node])  # put org master 2nd as will default in tie
            # apply the biggest masterid to the node in question
            node_master[node] = new_master
            # apply the biggest masterid to all nodes in question
            node_master[n] = new_master
    return node_master
            
params.node_master = near_by_simple(db, params.NODE, params.node_master, 75)  # 1619.62 sec #26 minutes

'near_by_simple' 2098.43 sec


In [38]:
save_dictionaries('OUT')  # 10.07 sec

'save_dictionaries' 5.90 sec


# Part 6
### Build simple intersections from small triangles 
irregrualr intersections 

In [44]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 84.76 sec


In [52]:
def closest_n(altGraph, nodeIsIntersection, loc, node, n):
    dl = []
    x, y = loc[node]  # starting position
    for l in altGraph[node]:
        # get coordinates of other end of each seg at start node
        if nodeIsIntersection[l[0]]:
            x1, y1 = loc[l[0]]
            d = distance(x, y, x1, y1)
            # add distance and node to list
            dl.append([d, l[0]])
    z = sorted(dl, key=itemgetter(0))
    # return closest N nodes with their distances
    return z[:n]

@timeDec
def triangle(folder, node_master, altGraph, nodeIsIntersection, triangle_dist=150, n=2):
    # load in the location dict
    loc = cpickle.load(open(os.path.join(folder, "loc.p"), "rb"))
    for node in node_master.keys():
        # for each node get the closest 2
        near = closest_n(altGraph, nodeIsIntersection, loc, node, n)
        # if the furthest near node is within tolerance
        if len(near) > 1:
            if near[-1][0] <= triangle_dist:
                # check can get to each in 1 hop
                # set of 2 closest is subset of nodes that are 1 hop in graph
                # clean up for double segs (vertical topography?)
                sn = set(n[1] for n in near)
                if len(sn) > 1:
                    a, b = set(n[1] for n in near)
                    if a in set(n[0] for n in altGraph[b]):
                        # if set(n[1] for n in near).issubset(set(n[0] for n in altGraph[node])):
                        # should be 1 master
                        for othernode in set(n[1] for n in near):
                            # go through those nodes
                            new_master = get_best_master(node_master,
                                node_master[othernode], node_master[node])  # put org master 2nd as will default in tie
                            # apply the biggest masterid to the node in question
                            node_master[node] = new_master
                            node_master[othernode] = new_master
    return node_master

params.node_master = triangle(params.FOLDER, params.node_master, params.altGraph, params.nodeIsIntersection, 150, 2) 

'triangle' 8.14 sec


In [53]:
save_dictionaries('OUT')  # 10.07 sec

'save_dictionaries' 6.04 sec


# Part 7
### update database intersections and build master segments


In [44]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 84.76 sec


In [55]:

@timeDec
def update_db_nodes(db, folder, node_table, node_master):
    # prep dict to write to csv
    x = [[node, node_master[node]] for node in node_master]
    # write out data
    write_csv(os.path.join(folder, "nodeMaster.csv"), x, ['nodeid', 'masterid'])
    # create temp_table
    db.query('drop table if exists nodeMaster; CREATE TABLE nodeMaster (nodeid numeric(10,0), masterid numeric(10,0))')
    
    raw_input(
        """PG Console \n\n \copy nodeMaster FROM '%s\ nodeMaster.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % folder)
    clear_output()

    db.query("""update %s
                set masterid = nm.masterid
                from nodemaster as nm
                where node.nodeid = nm.nodeid
                """ % node_table)
    db.query('drop table nodemaster')

update_db_nodes(db, params.FOLDER, params.NODE, params.node_master)  # 63.31 sec

Update sucessfull
Update sucessfull
'update_db_nodes' 63.31 sec


In [57]:
query_to_table(db, "select * from {} where masterid is not null limit 10".format(params.NODE))

,gid,objectid,nodeid,globalid,vintersect,geom,version,masterid,is_int
0,44,45,45,{D4F31166-CEBB-4F18-A03E-B27C751066FF},None,0101000020D7080000004DA3B4F1EA2B4100F86DEA06B9...,16d,31594,-1
1,3022,3081,3243,{8A14E9C5-615A-4F95-8A18-36A5C9816E0B},None,0101000020D708000080FA7782EB672C41001EEDA08BEF...,16d,13156,-1
2,3023,3082,3244,{869D1DEA-3194-4B0B-B4A2-5FF1C126E42B},None,0101000020D708000000F06807A7682C41005A8089E3F8...,16d,33637,-1
3,10623,10753,11242,{24ACDDDB-244C-4A2C-A86A-AF57782C3D64},None,0101000020D708000080B1A273555F2D410010E38DC1D2...,16d,13520,-1
4,18175,18371,19140,{393C21C5-5BC6-43C7-9123-BA745E2D2CCE},None,0101000020D708000080DA4664523B2E41002A2C4FC4B0...,16d,5267,-1
5,19984,20198,21081,{B541ACE9-E499-457E-A1A8-259497FAF249},None,0101000020D70800000071ECBF65FE2D41001830BE5AC8...,16d,22374,-1
6,19985,20199,21082,{F57C9103-3438-4C86-956F-7B421C57FEA7},None,0101000020D7080000004952AE62FF2D41003C36B967CF...,16d,891,-1
7,19986,20200,21083,{4AFAFB6F-3E3B-4935-91DC-5EF03360F57E},None,0101000020D7080000000B2C1940FA2D41003ABE11ADE4...,16d,39770,-1
8,19987,20201,21084,{A3A83FA7-034A-4A3F-B82E-F8381C15C716},None,0101000020D70800000036D05252FB2D4100B8BC6639EC...,16d,35970,-1
9,29865,30180,31515,{D7E76505-2B5B-4E9D-942E-99651814403E},None,0101000020D7080000000B00778D3A2F4100EAA2A843BF...,16d,15777,-1


In [62]:


@timeDec
def simple_from_to_masters(db, lion, node):
    print 'Updating Master From/To segments'
    # update the from nodes for simple streets
    # update the to nodes for simple streets

    # make the min master as from and the max master as to
    db.query("""create table tempFrom as (
                    select p1.mft, mn, mx from (
                    select mft, min(masterid) as mn from (
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidfrom::int = nodeid
                    where mft is not null and masterid is not null
                union
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidto::int = nodeid
                    where mft is not null and masterid is not null
                    ) as mn group by mft
                    ) as p1
                    join (

                    select mft, max(masterid) as mx from (
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidfrom::int = nodeid
                    where mft is not null and masterid is not null
                union
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidto::int = nodeid
                    where mft is not null and masterid is not null
                    ) as mx group by mft) as p2
                    on p1.mft = p2.mft
                    );
                    """.format(lion, node))
    # update lion with mfrom and to values
    db.query("""update %s as l
                    set masteridfrom = mn, masteridto= mx
                    from tempFrom
                    where l.mft = tempFrom.mft 
                    """ % lion)
    db.query("drop table tempFrom ")

simple_from_to_masters(db, params.LION, params.NODE)  # 111.35 sec

query_to_table(db, "select * from {} where street = 'QUEENS BOULEVARD' limit 10".format(params.LION))

Updating Master From/To segments
Update sucessfull
Update sucessfull
Update sucessfull
'simple_from_to_masters' 24.38 sec


,gid,street,safstreetn,featuretyp,segmenttyp,incexflag,rb_layer,nonped,trafdir,trafsrc,...,toleft,fromright,toright,join_id,shape_leng,geom,version,mft,masteridfrom,masteridto
0,148102,QUEENS BOULEVARD,None,0,R,None,R,D,W,DCP,...,0,0,0,4389401060000,51.2206821229,0105000020D70800000100000001020000000200000000...,16d,NaN,NaN,NaN
1,148456,QUEENS BOULEVARD,None,0,R,None,R,None,A,DOT,...,0,0,0,4389401050800,158.864631348,0105000020D70800000100000001020000000300000000...,16d,NaN,NaN,NaN
2,153884,QUEENS BOULEVARD,None,0,R,None,R,None,A,DCP,...,0,0,0,4389401050800,172.184700968,0105000020D70800000100000001020000000200000000...,16d,NaN,NaN,NaN
3,156357,QUEENS BOULEVARD,None,0,R,None,R,None,W,DOT,...,0,0,0,4389401061000,285.928469027,0105000020D70800000100000001020000000200000000...,16d,NaN,NaN,NaN
4,152286,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,60099,60010,60098,4389401000000,163.205061601,0105000020D70800000100000001020000000200000080...,16d,107797,18952,34543
5,178249,QUEENS BOULEVARD,None,0,R,None,R,None,W,DOT,...,0,0,0,4389401061000,77.1858173897,0105000020D70800000100000001020000000200000000...,16d,NaN,NaN,NaN
6,179285,QUEENS BOULEVARD,None,0,R,None,R,None,A,DCP,...,0,0,0,4389401050800,151.395118140,0105000020D70800000100000001020000000200000000...,16d,NaN,NaN,NaN
7,152308,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,55099,55020,55098,4389401290000,47.9024145777,0105000020D70800000100000001020000000200000000...,16d,99036,16047,40195
8,151376,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,49099,49002,49098,4389401290000,228.423340066,0105000020D70800000100000001020000000200000000...,16d,98737,6329,12579
9,121024,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,49001,49000,49000,4389401290000,31.6511620793,0105000020D70800000100000001020000000200000000...,16d,98737,6329,12579


In [63]:

@timeDec
def get_mfts(db, lion):
    # go through list fo mfts
    data = db.query(
        """select mft from {0} where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
            and mft is not null and street != 'UNNAMED STREET' and street not like '%{1}%' group by mft
        """.format(lion, 'PED OVPS'))
    # and (masteridfrom is null or masteridto is null)
    local_mfts = [i[0] for i in data[0]]
    return local_mfts


l_mfts = get_mfts(db, params.LION)  # 0.56 sec

'get_mfts' 0.93 sec


In [65]:
@timeDec
def make_c_master_seg_table(db, node_table, srid):
    print 'Making new table'

    db.query("drop table if exists c_master_segs")
    db.query("""CREATE TABLE c_master_segs ( 
              mft    INTEGER,
              street    varchar(100),
              segmentid    varchar(10),
              geom    geometry(LINESTRING, %i),
              masteridfrom    INTEGER, 
              masteridto    INTEGER,
              seg_cnt INTEGER
              );""" % srid)

    db.query("drop table if exists c_master_node")
    db.query("""create table c_master_node as (
                select distinct masterid,
                        st_centroid(st_union(geom)) as geom,
                        st_x(st_centroid(st_union(geom))) as centerX,
                        st_y(st_centroid(st_union(geom))) as centerY,
                        count(*)::int as nodes
                from %s 
                where masterid !=0
                group by  masterid
                )""" % node_table)

make_c_master_seg_table(db, params.NODE, params.SRID)  # 1.58 sec
query_to_table(db, "select * from c_master_node limit 10")

Making new table
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
'make_c_master_seg_table' 1.58 sec


,masterid,geom,centerx,centery,nodes
0,1,0101000020D708000000CF491C053630410046E6BED17C...,1062405.110501,208794.218213,1
1,3,0101000020D7080000804E630030872F4100F6A4BEF77D...,1033112.000758,241598.968088,1
2,4,0101000020D708000080648914348B2E4100BC772A04E7...,1000858.040111,154848.520736,1
3,5,0101000020D70800004091EA2BB3E52F41008DDA9B8096...,1045209.585774,217808.076100,2
4,7,0101000020D708000000905F856A9F2F4100285E0F9128...,1036213.260495,214290.132504,1
5,8,0101000020D708000080130B89311D2F410076F1DBBB12...,1019544.767663,189015.482394,1
6,9,0101000020D70800000085B3F5D0442C4100BE560D8620...,926312.479885,132112.756513,1
7,10,0101000020D70800008055BE7CDE4B2F41003000B9048F...,1025519.243640,242144.590332,1
8,11,0101000020D7080000805512E9314C2E4100569B985A82...,992792.955218,184395.324515,1
9,12,0101000020D708000080022EA4D2572D4100D4D14496F4...,961513.320664,171666.783603,1


In [67]:
@timeDec
def mft_round2(db, lion):
    print 'Updating mft (round 2)'
    # make temp table for segs with same from and to masters and update lion with results
    db.query("""drop table if exists _temp_mft_update_;

                create table _temp_mft_update_ as (
                select min(mft) as mft,masteridfrom, masteridto
                from {0}
                where mft is not null and masteridfrom is not null and masteridto is not null
                group by masteridfrom, masteridto);

                update {0} as l
                set mft = t.mft
                from _temp_mft_update_ as t
                where l.masteridfrom = t.masteridfrom
                and l.masteridto = t.masteridto;

                drop table if exists _temp_mft_update_;
                """.format(lion))

# TODO seems to be missing master segs for park ave BK, but the data is valid in lion and lion tamed
mft_round2(db, params.LION)  # 63.77 sec

query_to_table(db, "select * from {} where street = 'QUEENS BOULEVARD' and rb_layer in ('B', 'G') limit 10".format(params.LION))

Updating mft (round 2)
Update sucessfull
'mft_round2' 33.90 sec


,gid,street,safstreetn,featuretyp,segmenttyp,incexflag,rb_layer,nonped,trafdir,trafsrc,...,toleft,fromright,toright,join_id,shape_leng,geom,version,mft,masteridfrom,masteridto
0,126379,QUEENS BOULEVARD,None,0,G,None,G,D,T,DOT,...,0,0,0,4389401000000,24.7217396370,0105000020D70800000100000001020000000200000000...,16d,83014,4313,4313
1,176853,QUEENS BOULEVARD,None,0,F,None,G,None,W,DCP,...,0,0,0,4389401000000,69.1987705367,0105000020D70800000100000001020000000200000080...,16d,120654,19004,24241
2,156226,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,0,0,0,4389401000000,44.9001296632,0105000020D70800000100000001020000000200000080...,16d,102469,34054,42177
3,132711,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,69099,69030,69098,4389401000000,144.465926927,0105000020D70800000100000001020000000200000080...,16d,108663,26144,32520
4,120996,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,69015,69012,69018,4389401000000,35.1878456639,0105000020D70800000100000001020000000200000080...,16d,108663,26144,32520
5,128711,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,69025,69020,69028,4389401000000,109.281154413,0105000020D70800000100000001020000000200000000...,16d,108663,26144,32520
6,155989,QUEENS BOULEVARD,None,0,G,None,G,None,T,DOT,...,88011,88000,88010,4389401000000,132.530073156,0105000020D70800000100000001020000000200000080...,16d,109966,37027,40762
7,119120,QUEENS BOULEVARD,None,0,G,None,G,D,T,DOT,...,0,0,0,4389401000000,212.006073287,0105000020D70800000100000001020000000200000000...,16d,83014,4313,4313
8,116137,QUEENS BOULEVARD,None,0,G,None,G,D,T,DOT,...,0,0,0,4389401000000,45.5493350297,0105000020D70800000100000001020000000200000080...,16d,83014,4313,4313
9,122058,QUEENS BOULEVARD,None,0,G,None,G,D,T,DOT,...,0,0,0,4389401000000,24.7218998797,0105000020D70800000100000001020000000200000000...,16d,83014,4313,4313


In [68]:
save_dictionaries('OUT')  # 10.07 sec

'save_dictionaries' 53.30 sec


# Part 8
### update database with final data


In [69]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 8.70 sec


In [94]:
def get_simple_segs(db, node_table):
    data = db.query("""select mft, st_astext(ST_LineMerge(geom)), simple.masterid,simple2.masterid
                from lion
                ,(
                    select n.nodeid::int, n.masterid
                    from {0} as n
                    join c_master_node
                    on n.masterid = c_master_node.masterid
                    where n.geom =c_master_node.geom
                ) as simple
                ,(
                    select n.nodeid::int, n.masterid
                    from {0} as n
                    join c_master_node
                    on n.masterid = c_master_node.masterid
                    where n.geom =c_master_node.geom
                ) as simple2
                where nodeidfrom::int = simple.nodeid and nodeidto::int = simple2.nodeid
                """.format(node_table))
    simple_mfts = set()
    line = []
    for row in data[0]:
        mft, mft_line, mfrom, mto = row
        simple_mfts.add(mft)
        line.append([mft, mft_line, mfrom, mto])
    return simple_mfts, line


def get_compex_segs(db, lion):
    # create table with a count of verticies where are more than 2 so they can be remapped
    db.query("""drop table if exists complexSegs; create table complexSegs as (
                    select mft, max(pos)
                    from (
                            select mft, 
                            (ST_DumpPoints(ST_LineMerge(ST_Union(geom)))).path[1] as pos
                            from %s
                            group by  mft
                                    )as mx
                    where mft is not null
                    group by mft
                    having max(pos) > 2 ) """ % lion)
    # store the mfts that need to be remapped
    data = db.query("select mft from complexSegs")
    to_do = set()
    for mft in data[0]:
        to_do.add(mft[0])
    return to_do


def coord_from_master_qry(db, coordFromMaster):
    
    data = db.query("select  masterid, st_x(geom), st_y(geom) from c_master_node")
    for row in data[0]:
        master, x, y = row
        coordFromMaster[master] = [x, y]
    return coordFromMaster


def get_geom(db, lion):
    d = {}
    data = db.query("""select mft, ST_asText(ST_LineMerge(ST_Union(geom))) as g
                ,masteridfrom, masteridto
                from %s
                where mft is not null and masteridfrom is not null and masteridto is not null
                group by mft, masteridfrom, masteridto;
                """ % lion)
    for output in data[0]:
        mft, geom, mfrom, mto = output
        d[mft] = [geom, mfrom, mto]
    return d

def simple_new_ends(coordFromMaster, p1, p2):
    return "LINESTRING (%s %s, %s %s)" % (str(coordFromMaster[p1][0]), str(coordFromMaster[p1][1]),
                                          str(coordFromMaster[p2][0]), str(coordFromMaster[p2][1]))


def simplify_coord(wkt):
    x = str(wkt.split("(")[1]).split(")")[0]
    if "," not in x:
        x = x.replace("[", '').replace(" ", ",")
    return [float(i) for i in x.split(",")]

def clean(pt):
    if type(pt) != list:
        return [simplify_coord(pt)[0], simplify_coord(pt)[1]]
    else:
        return pt

def coord_to_wkt(coord):
    coord = str(coord)[1:-1].replace(",", "")
    return "POINT(" + coord + ")"

def get_new_ends(db, geom, lion, coordFromMaster):
    # store midles
    d = {}  # {mft: [1st vertex, 2nd vertex...]}
    # get geom without first or last vetex
    data = db.query("""select t.mft, pos, st_astext(geom )
                    from (
                        select mft,
                        (ST_DumpPoints(ST_LineMerge(ST_Union(geom)))).path[1] as pos
                        , (ST_DumpPoints(ST_LineMerge(ST_Union(geom)))).geom as geom
                        from %s
                        where masteridfrom is not null and masteridto is not null 
                        group by mft
                    ) as t
                    join complexSegs as cs 
                    on t.mft=cs.mft
                    where pos > 1 and pos < cs.max""" % lion)
    for row in data[0]:
        mft, pos, coord = row
        if mft not in d.keys():
            d[mft] = [coord]
        else:
            d[mft].append(coord)
    # find closest end point to from or to
    for mft in d.keys():
        start = clean(d[mft][0])
        end = clean(d[mft][-1])
        frm = clean(coordFromMaster[geom[mft][1]])
        to = clean(coordFromMaster[geom[mft][2]])
        # check if only 1 vertex left and rebuild collection
        if start == end:
            d[mft] = [coord_to_wkt(frm), coord_to_wkt(start), coord_to_wkt(to)]
        else:
            # check if start is closer to from or to
            dist = distance(start[0], start[1], frm[0], frm[1])  # SF
            if distance(start[0], start[1], to[0], to[1]) < dist:  # ST
                dist = distance(start[0], start[1], to[0], to[1])
                # start is closer to to, but need to check that start/to is closer that end/to
                if distance(end[0], end[1], to[0], to[1]) < dist:  # ET
                    d[mft] = [coord_to_wkt(frm)] + d[mft] + [coord_to_wkt(to)]
                else:
                    d[mft] = [coord_to_wkt(to)] + d[mft] + [coord_to_wkt(frm)]  # ST
            else:  # SF
                # start is closer to frm, but need to check that start/frm is closer that start/frm
                if distance(end[0], end[1], frm[0], frm[1]) < dist:  # EF
                    d[mft] = [coord_to_wkt(to)] + d[mft] + [coord_to_wkt(frm)]  # ST
                else:
                    d[mft] = [coord_to_wkt(frm)] + d[mft] + [coord_to_wkt(to)]  # SF
        # clean up and make line
        # print 'Finishing mft %i' %(mft)
        d[mft] = line_from_pts(d[mft]).replace("'", "")
    return d

def line_from_pts(wkt_list):
    out = 'LINESTRING('

    def simple(pt):
        return [float(j) for j in pt.split("(")[1].split(")")[0].split(" ")]

    for i in wkt_list:
        out = out + str(simple(i)[0]) + " " + str(simple(i)[1]) + ", "
    return out[:-2] + ")"

In [95]:



@timeDec
def remap(db, folder, node_table, lion, srid, coordFromMaster):
    # change insert 1 row at a time to csv io model
    simple_mfts, list_c_master_segs = get_simple_segs(db, node_table)
    complex_mfts = get_compex_segs(db, lion)
    # get coordinates for new end points
    params.coordFromMaster = coord_from_master_qry(db, params.coordFromMaster)
    # get basic info for corridors
    geom = get_geom(db, lion)
    counter = 0
    print 'found %i segments that are simple and do not need remapping' % (len(list_c_master_segs))
    todo = []
    for mft in geom.keys():
        if mft in simple_mfts:
            # no change needed
            pass
        elif mft in complex_mfts:
            # check if simple or complex change
            pass
        else:
            todo.append(mft)
    for mft in todo:
        geo, mfrom, mto = geom[mft]
        if 'MULTILINESTRING' in geo:
            pass
        elif mft and mfrom and mto:
            # simple straight  line, just use master from/to as new coordinates
            list_c_master_segs.append([mft, simple_new_ends(coordFromMaster, mfrom, mto), mfrom, mto])

    complex_geom = get_new_ends(db, geom, lion, coordFromMaster)
    for mft in complex_geom.keys():
        new_geom, mfrom, mto = complex_geom[mft], geom[mft][1], geom[mft][2]
        # change insert 1 row at a time to csv io model
        if 'MULTILINESTRING' in new_geom:
            pass
        else:
            list_c_master_segs.append([mft, new_geom, mfrom, mto])
            counter += 1
    db.query("drop table complexSegs")

    print 'done re-mapping %i sgements' % counter
    write_csv(os.path.join(folder, "tempMaster.csv"), list_c_master_segs, ['mft', 'mftLine', 'mfrom', 'mto'])
    db.query("drop table if exists tempMaster; CREATE TABLE tempMaster (mft int, mftLine text, mfrom int, mto int)")
    raw_input(
        """PG Console \n\n \copy tempMaster FROM '%s\\tempMaster.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % folder)
    clear_output()
    db.query("""insert into c_master_segs (mft, geom, masteridfrom, masteridto)
                ((select mft, st_geomfromtext(mftLine, %i), mfrom, mto from tempMaster
                where mft is not null group by mft, mftline, mfrom, mto))""" % srid)
    db.query("drop table tempMaster")

remap(db, params.FOLDER, params.NODE, params.LION, params.SRID, params.coordFromMaster)  # 122.62 sec

query_to_table(db, "select * from c_master_segs limit 10")

Update sucessfull
Update sucessfull
'remap' 134.05 sec


,mft,street,segmentid,geom,masteridfrom,masteridto,seg_cnt
0,1,None,None,0102000020D708000002000000DA33320756B52E41F7E5...,35103,28682,None
1,2,None,None,0102000020D708000002000000F5CCFD75DAB42E41F721...,421,35103,None
2,3,None,None,0102000020D70800000400000053D0ED75DAB42E413B18...,421,43946,None
3,4,None,None,0102000020D7080000190000005D15A8ADEB692E41B491...,14378,36788,None
4,5,None,None,0102000020D70800000600000015C440F7846D2E41BBB6...,14378,21044,None
5,6,None,None,0102000020D70800000200000015C440F7846D2E41BBB6...,14378,14378,None
6,7,None,None,0102000020D70800000200000000FE91A198E32B41F423...,31386,43033,None
7,8,None,None,0102000020D7080000020000008107EA947AE02B4116D0...,39547,43033,None
8,9,None,None,0102000020D708000002000000C15433EB23E12B41C1FF...,43033,43033,None
9,10,None,None,0102000020D708000003000000EE2763D468E32B41C689...,22674,22674,None


In [98]:
def update_c_master_segs(db, lion):
    db.query("""update c_master_segs as ms
                    set segmentid = q.segmentid,street = q.street
                    from 
                    (
                    select street, lion.segmentid , lion.mft
                    from {0} 
                    , (
                            select mft, segmentid, max(st_length(geom)) as ml
                            from {0}
                            group by mft, segmentid, st_length(geom)
                    ) as mln
                    where {0}.segmentid = mln.segmentid
                    group by street, {0}.segmentid , {0}.mft
                    ) as q
                    where ms.mft=q.mft
                """.format(lion))
    db.query("""update c_master_segs as ms
                    set seg_cnt = cnt
                    from 
                    (
                            select mft, count(*) as cnt from %s group by mft
                    ) as q
                    where ms.mft=q.mft;
                """ % lion)

update_c_master_segs(db, params.LION)


Update sucessfull
Update sucessfull


In [100]:
def stringify(int_to_sting, length):
    int_to_sting = str(int_to_sting)
    if len(int_to_sting) == length:
        return int_to_sting
    else:
        return stringify("0" + int_to_sting, length)
    
@timeDec
def update_roadbeds(db, folder, lion, tbl_rpl):
    data = db.query("select segmentid, mft from %s where mft is not null" % lion)
    seg_mft = {}
    to_test = set()
    for i in data[0]:
        seg, mft = i
        seg_mft[seg] = mft

    data = db.query("""select segmentid, other from (
                    select segmentid, segmentidg as other 
                    from {0} as l
                    join {1} on l.segmentid::int = segmentidr 
                    union
                    select segmentid, segmentidr as other 
                    from {0} as l
                    join {1} on l.segmentid::int = segmentidg
                    ) as base group by segmentid, other
                     """.format(lion, tbl_rpl))
    for row in data[0]:
        seg1, seg2 = row[0], stringify(row[1], 7)
        if seg1 in seg_mft.keys():
            # seg2 might not be in the dict
            if seg2 not in seg_mft.keys():
                # add rb to dict with centerline mft
                seg_mft[seg2] = seg_mft[seg1]
        elif seg2 in seg_mft.keys():
            # seg2 might not be in the dict
            if seg1 not in seg_mft.keys():
                # add rb to dict with centerline mft
                seg_mft[seg1] = seg_mft[seg2]
        else:
            to_test.add(seg1)
            to_test.add(seg2)

    print 'Still missing %i segments\n' % (len(to_test))
    out = [[key, seg_mft[key]] for key in seg_mft]

    write_csv(os.path.join(folder, "tempMaster.csv"), out, ['Segmentid', 'MFT'])

    db.query("drop table if exists tempMaster; CREATE TABLE tempMaster (Segmentid int, MFT int)")

    raw_input(
        """PG Console \n\n \copy tempMaster FROM '%s\\tempMaster.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % folder)
    clear_output()
    db.query("""update %s as l
                    set mft = t.mft
                    from tempMaster as t
                    where l.segmentid::int = t.segmentid
                    and l.mft is null""" % lion)

    db.query("drop table tempMaster")

    db.query("""update %s as l
                    set masteridfrom = m.masteridfrom, masteridto = m.masteridto
                    from c_master_segs as m
                    where l.mft = m.mft""" % lion)

update_roadbeds(db, params.FOLDER, params.LION, params.RPL)  # 1302.61 sec
query_to_table(db, "select * from c_master_segs limit 10")

Update sucessfull
Update sucessfull
Update sucessfull
'update_roadbeds' 818.95 sec


,mft,street,segmentid,geom,masteridfrom,masteridto,seg_cnt
0,6642,DOLSON PLACE,0110274,0102000020D708000002000000334E43AC529A2C4196CC...,1851,1851,1
1,33477,MAIN STREET,0024084,0102000020D708000002000000FF1AFCD3A91D2E410410...,40693,23134,1
2,54635,2 AVENUE,0038055,0102000020D7080000020000007FB74D6AF07E2E41FDBF...,18342,19015,1
3,55959,LENOX AVENUE,0038847,0102000020D708000002000000FD37F7BD5C7F2E41FA0F...,34721,16422,1
4,75251,91 AVENUE,0051059,0102000020D708000002000000F91BE5C727332F41F5D1...,15654,8029,1
5,98160,47 AVENUE,0067359,0102000020D70800000200000007EC8B3C75942E41FDA5...,31806,2756,1
6,101655,23 AVENUE,0068780,0102000020D708000002000000EE50A005E7BA2E41F893...,17765,23430,2
7,102290,PARK AVENUE,0069206,0102000020D70800000200000087ED9BFB5B8F2E410020...,13762,24216,1
8,102804,GERARD AVENUE,0069864,0102000020D7080000020000007C1C9B2F10A92E41FB0D...,41790,27682,1
9,104890,GERARD AVENUE,0071596,0102000020D708000002000000A722B2C356AF2E41F1FF...,43515,11657,1


In [104]:

@timeDec
def update_roadbed_nodes(db, node_table, tbl_rpl):
    db.query("""create table m as (
                select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.r_tond
                    join {0} as nn
                        on g_tond = nn.nodeid
                    where n.masterid is null and nn.masterid is not null
                union
                    select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.g_tond
                    join {0} as nn
                        on r_tond = nn.nodeid
                    where n.masterid is null and nn.masterid is not null
                union
                    select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.r_frnd
                    join {0} as nn
                        on g_frnd = nn.nodeid
                    where n.masterid is null and nn.masterid is not null
                union
                    select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.g_frnd
                    join {0} as nn
                        on r_frnd = nn.nodeid
                    where n.masterid is null and nn.masterid is not null)
                """.format(node_table, tbl_rpl))

    for i in range(1, 5):
        db.query("""update %s as n
                    set masterid = m.masterid
                    from m
                    where n.nodeid=m.nodeid and n.masterid is null""" % node_table)
    db.query("drop table m")
    db.query("""update {0} as n
                    set masterid  = nn.masterid
                    from (
                        select distinct r,g from (
                        select r_frnd as r, g_frnd as g
                        from {1}
                        union
                        select r_tond as r, g_tond as g
                        from {1}
                        ) as passthrough
                    )as nl, {0} as nn
                    where n.nodeid::int = r::int
                    and  g::int = nn.nodeid::int
                    and n.masterid is null and nn.masterid is not null
                """.format(node_table, tbl_rpl))
    db.query("""update {0} as n
                    set masterid  = nn.masterid
                    from (
                        select distinct r,g from (
                        select r_frnd as r, g_frnd as g
                        from {1}
                        union
                        select r_tond as r, g_tond as g
                        from {1}
                        ) as passthrough
                    )as nl, {0} as nn
                    where n.nodeid::int = g::int
                    and  r::int = nn.nodeid::int
                    and n.masterid is null and nn.masterid is not null
                """.format(node_table, tbl_rpl))
    
update_roadbed_nodes(db, params.NODE, params.RPL)
query_to_table(db, "select * from {0} limit 10".format(params.NODE))

Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
'update_roadbed_nodes' 3.35 sec


,gid,objectid,nodeid,globalid,vintersect,geom,version,masterid,is_int
0,67074,67808,70627,{BBFF7168-B256-47B1-B9D6-D92E86F5E022},None,0101000020D70800000036995065672E41001A66499FA0...,16d,30485,-1
1,67080,67814,70633,{74988050-9459-4768-858B-F8863DCE0EA1},None,0101000020D708000000BA2B6824992E4100FA73D91867...,16d,3952,-1
2,67497,68469,71315,{AA81AE90-B5E6-4632-B447-C4061FBC12A0},None,0101000020D7080000007237DE4EB62E41000E7F1FFF2C...,16d,10721,-1
3,73826,74883,77775,{D970E10F-E5D2-40F8-86CB-2CB309F7487F},None,0101000020D7080000000268CD34022C41000A5F2F6040...,16d,28276,-1
4,73827,74884,77776,{73DE3AF5-145A-4FD2-A208-D1B6F88BF76E},None,0101000020D708000080CAA4DE6E012C41008647FF2420...,16d,42142,-1
5,75617,76702,79617,{E6D0B459-3FC8-49BA-B285-4C248745E785},None,0101000020D7080000003C7C31B4222E4100CA479735B9...,16d,23952,-1
6,75669,76755,79670,{9B70EA5E-3BB3-4C5E-9E61-F7DCD533B4DD},None,0101000020D7080000009316A3E4332E41004E114EE390...,16d,42403,-1
7,78772,79884,82839,{79421E6E-8056-4A50-964A-0CFD22B4C6C5},None,0101000020D708000000872DA3881830410052DC4DA814...,16d,4867,-1
8,78806,79919,82874,{FA76A9A9-A755-40C5-9A27-D8AF94F934E5},None,0101000020D708000000EA3C8B45ED2F4100B6CB3C7E20...,16d,12644,-1
9,80966,82108,85079,{D4EF14B8-6EAE-4D5C-AE6E-1F8775584F15},None,0101000020D708000080544B4B7C792F4100C6201A13CD...,16d,44345,-1


In [117]:

def make_centerline_clean(db, lion, node_table, version):
    db.query("""drop table if exists C_Lion_{0}; create table C_Lion_{0} as (
                    select distinct *
                    from {1}
                    where rb_layer in ('B', 'G')
                            and featuretyp in ('0', '6', 'C')
                            and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                            or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                            and street != 'UNNAMED STREET'
                            and street not like '%PED OVPS%'
                            and mft is not null
                    )""".format(version, lion))

    db.query("""drop table if exists C_Lion_Nodes_{0}; create table C_Lion_Nodes_{0} as (
                    select distinct *
                    from {1}
                    where is_int =-1
                    )""".format(version, node_table))
    print '\nDONE!\n'

make_centerline_clean(db, params.LION, params.NODE, params.VERSION)

query_to_table(db, "select * from  C_Lion_{0} limit 10".format(params.VERSION))

Update sucessfull
Update sucessfull

DONE!



,gid,street,safstreetn,featuretyp,segmenttyp,incexflag,rb_layer,nonped,trafdir,trafsrc,...,toleft,fromright,toright,join_id,shape_leng,geom,version,mft,masteridfrom,masteridto
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,...,699,596,716,2251001000000,396.030946672,0105000020D70800000100000001020000000200000000...,16d,114414,5429,27418
1,2,WEST 192 STREET,None,0,U,None,B,None,A,DOT,...,98,63,99,2798401000000,279.360514147,0105000020D70800000100000001020000000200000080...,16d,116923,4565,43565
2,3,UNION AVENUE,None,0,U,None,B,None,W,DOT,...,1079,1016,1084,2728001000000,618.327132916,0105000020D70800000100000001020000000200000000...,16d,113419,16598,40962
3,4,UNION AVENUE,BEHAGEN PLAYGROUND,0,U,None,B,None,W,DOT,...,0,0,0,21279501000000N,618.327132916,0105000020D70800000100000001020000000200000000...,16d,113419,16598,40962
4,5,DELAFIELD AVENUE,None,6,U,None,B,None,T,DOT,...,4645,4600,4664,2187601000000,670.281037291,0105000020D70800000100000001020000000200000080...,16d,107458,35120,16898
5,6,WEST 252 STREET,None,0,U,None,B,None,T,DCP,...,0,0,0,2814401000000,16.8975586904,0105000020D70800000100000001020000000200000080...,16d,107486,30631,30631
6,7,CLAREMONT PARKWAY,None,0,U,None,B,None,T,DOT,...,439,400,416,2147601000000,252.183072235,0105000020D70800000100000001020000000200000080...,16d,114474,21781,35574
7,8,TOPPING AVENUE,None,0,U,None,B,None,W,DOT,...,1849,1800,1838,2718001000000,291.757016214,0105000020D70800000100000001020000000200000080...,16d,114550,32533,3194
8,9,EAST 175 STREET,None,0,U,None,B,None,T,DOT,...,299,260,298,2255001000000,249.533157113,0105000020D70800000100000001020000000200000080...,16d,114552,32533,38400
9,10,WEST 234 STREET,None,0,U,None,B,None,T,DOT,...,298,281,299,2805201000000,261.095117693,0105000020D70800000100000001020000000200000080...,16d,118339,37585,34131


In [106]:
@timeDec
def add_boroughs_to_intersections(db, version, boroughs):
    nodes = 'C_Lion_nodes_' + version
    print 'Updating boroughs for nodes\n'
    db.query("""alter table {0} drop column if exists lboro;
                    alter table {0} drop column if exists rboro;
                    alter table {0} add column lboro int;
                    alter table {0} add column rboro int;
                    """.format(nodes))

    db.query("""update %s as n
                    set lboro = borocode
                    from %s as b
                    where st_dwithin(n.geom, b.geom, 5)
                    and lboro is null;""" % (nodes, boroughs))
    db.query("""update %s as n
                    set rboro = borocode
                    from %s as b
                    where st_dwithin(n.geom, b.geom, 5)
                    and rboro is null;""" % (nodes, boroughs))

add_boroughs_to_intersections(db, params.VERSION, params.BOROUGHS)  # 400.35 sec
query_to_table(db, "select * from  C_Lion_nodes_{0} limit 10".format(params.VERSION))

Updating boroughs for nodes

Update sucessfull
Update sucessfull
Update sucessfull
'add_boroughs_to_intersections' 404.73 sec


,gid,objectid,nodeid,globalid,vintersect,geom,version,masterid,is_int,lboro,rboro
0,19398,19608,20457,{8261840E-D804-402F-8C5A-393D9683ECF8},None,0101000020D708000000F8C51515072E41000219965358...,16d,12874,-1,1,1
1,19399,19609,20458,{845C1D1E-0B3C-428C-B164-DF1B2E3AB97D},None,0101000020D708000000C437DD69002E41005C9906376A...,16d,9227,-1,1,1
2,19400,19610,20459,{1E8251B6-9BB6-4880-9D30-0B19BCDAA0D1},None,0101000020D7080000803207D010012E4100C2A18B436A...,16d,9227,-1,1,1
3,19401,19611,20460,{F24FCDFB-E12F-403C-B4D8-64C14E8DB024},None,0101000020D708000000C0FA7418012E4100EA80572E6D...,16d,9227,-1,1,1
4,19402,19612,20461,{54334742-26A1-4C93-9463-172710E417A1},None,0101000020D708000000F51E0719012E4100EEBD904271...,16d,9227,-1,1,1
5,19403,19613,20462,{609A42C1-9E6A-4BD7-984E-225408619A6A},None,0101000020D708000080F594EC15012E4100F462B9D478...,16d,29573,-1,1,1
6,19404,19614,20463,{B672B4EF-6B9E-4585-91AE-1295E6124288},None,0101000020D7080000808199D3FD012E4100B6060B4B71...,16d,9227,-1,1,1
7,19405,19615,20464,{95EDF418-8D53-4A96-9471-4C05FD856CC3},None,0101000020D708000000536ECBC7022E4100BA252EF474...,16d,25716,-1,1,1
8,19406,19616,20465,{BBABBDC4-FB70-453E-9B22-A655BF790AB8},None,0101000020D70800008097830B6A032E4100BE29439478...,16d,38882,-1,1,1
9,19407,19617,20466,{9C529A87-92BF-4D5C-A5C9-299EFF4B85F4},None,0101000020D708000000984B39FD032E41001814A8A37C...,16d,1588,-1,1,1


In [127]:
def get_nodes(db, node_table):
    data = db.query(
        "select count(*) from (select distinct masterid from %s where masterid is not null) as t" % node_table)
    return data[0][0]

@timeDec
def make_lookup(db, node_table, lion_table, ver):
    db.query("drop table if exists c_lion_node_lookup")

    db.query('''create table c_lion_node_lookup as (
                select nodeid, masterid, mft, segmentid
                from %s as l
                join %s as n
                 on l.nodeidfrom::int = n.nodeid
                union
                select nodeid, masterid, mft, segmentid
                from %s as l
                join %s as n
                 on l.nodeidto::int = n.nodeid
                ) ''' % (lion_table, node_table, lion_table, node_table))
    # index lookup
    version = 'C_Lion_nodes_'+ver
    index_list = ["drop index if exists nd_IDX; ", "drop index if exists master_IDX;", "drop index if exists mft_IDX;",
                  "drop index if exists seg_IDX;", "drop index if exists lt_node_IDX",
                  "CREATE INDEX nd_IDX ON c_lion_node_lookup (nodeid);",
                  "CREATE INDEX master_IDX ON c_lion_node_lookup (masterid);",
                  "CREATE INDEX mft_IDX ON c_lion_node_lookup (mft);",
                  "CREATE INDEX seg_IDX ON c_lion_node_lookup (segmentid);",
                  "CREATE INDEX lt_node_IDX ON %s (nodeid);" % version]
    for idx in index_list:
        db.query(idx)

make_lookup(db, params.NODE, params.LION, params.VERSION)  # 122.01 sec

Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
'make_lookup' 13.42 sec


In [124]:
def get_mileage(db, lion):
    miles = {}
    for b in range(1, 6):
        data = db.query("""select sum(st_length(geom))/5280 from (
                        select distinct on (segmentid) geom from {0}
                        where
                            rb_layer in ('B', 'G')
                            and featuretyp in ('0', '6', 'C')
                            and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                            or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                            and street != 'UNNAMED STREET'
                            and street not like '%PED OVPS%'
                            and mft is not null
                            and (lboro = {1} or rboro = {1} )
                        group by segmentid, geom
                    ) as singles""".format(lion, b, b))
        miles[b] = data[0][0][0]
    data = db.query("""select sum(st_length(geom))/5280 from ( 
                    select distinct on (segmentid) geom from {0}
                    where
                        rb_layer in ('B', 'G')
                        and featuretyp in ('0', '6', 'C')
                        and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                        or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                        and street != 'UNNAMED STREET'
                        and street not like '%PED OVPS%'
                        and mft is not null
                    group by segmentid, geom
                ) as singles""".format(lion))
    miles[0] = data[0][0][0]
    return miles

miles = get_mileage(db, params.LION)

In [136]:
print '-----------------------------------------------------------------------------------'
print 'LION (Version {0}) files:\nBasic ({1}, {2}), \nUgly(c_master_segs, c_master_node), \
\nCenterline(C_Lion{0}, C_Lion_Nodes_{0})'.format(
    params.VERSION, params.LION, params.NODE)
print '-----------------------------------------------------------------------------------'
for i in miles:
    if i > 0:
        print 'Borough %i: %4.2f Miles of Centerline Non-Highway Roadway' % (i, miles[i])
    else:
        print 'Citywide: %4.2f Miles of Centerline Non-Highway Roadway' % (miles[i])
print '%i Intersections' % (get_nodes(db, params.NODE))
print '-----------------------------------------------------------------------------------'

-----------------------------------------------------------------------------------
LION (Version 16d) files:
Basic (lion, node), 
Ugly(c_master_segs, c_master_node), 
Centerline(C_Lion16d, C_Lion_Nodes_16d)
-----------------------------------------------------------------------------------
Citywide: 6196.96 Miles of Centerline Non-Highway Roadway
Borough 1: 566.11 Miles of Centerline Non-Highway Roadway
Borough 2: 814.70 Miles of Centerline Non-Highway Roadway
Borough 3: 1556.63 Miles of Centerline Non-Highway Roadway
Borough 4: 2262.70 Miles of Centerline Non-Highway Roadway
Borough 5: 1001.70 Miles of Centerline Non-Highway Roadway
42112 Intersections
-----------------------------------------------------------------------------------
